In [22]:
import pandas as pd 
import re as re 
import numpy as np
import time 
import mygene

pd.set_option('display.max_colwidth', -1)
pd.set_option('mode.chained_assignment', 'warn')

col_names = ['chromosome', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'] # For df object
#df = data frame (object of the Homo Sapians file)
df = pd.read_csv('Homo_sapiens.GRCh38.85.gff3.gz', compression='gzip',
                         sep='\t', comment='#', low_memory=False,
                         header=None, names=col_names)

# Old Functions

In [23]:
# Made this function in a better way below 
# def closestGeneBody(chrNum, site, window):
#     out = ['No Result', 9**90] # This is just a random num that will be replace by func below. 
#     geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
#     for index, row in geneDF.iterrows():
#         if inRange(row['start'], site, window):
#             if isNearBody(row['start'], row['end'], site, window):
#                 if distanceBody(row['start'], row['end'], site) < out[1]:
#                     out[0] = row['gene_name']
#                     out[1] = distanceBody(row['start'], row['end'], site)
#         else: 
#             return(out)
#     # Returns closest gene and its distance from body. 
#     return(out)

# def closestGenesNsitesBody(df, window):
#     count = 0 
#     geneL = []
# #     for index, row in df.iterrows():
# #         x = closestGeneBody(row['chromosome'], row['BindingSite'], window)
# #         geneL.append(x[0])
# #         count = count + 1
#     for row in zip(df['chromosome'], df['BindingSite']):
#         x = closestGeneBody(row[0], row[1], window)
#         geneL.append(x[0])
#         count = count + 1
#     return(count, geneL)

# finds unique in list | Might use later 
# def unique(list1): 
#     unique_list = [] 
#     for x in list1: 
#         if x not in unique_list: 
#             unique_list.append(x) 
#     return unique_list

# A. Estimating number of sites to be sampled randomly

# <font color=#4286f4>Setting up my DataFrame as variable: genes/chromosome</font>

In [24]:
geneAnnotations = df[df.type.isin(['gene'])].copy()
chromosome = df[df.type.isin(['chromosome'])].copy()

# <font color=#4286f4>Adding Columns to geneAnnotations: gene_symbol, gene_ensID, length</font>

In [25]:
# ONLY NEEDS TO BE DONE ONCE!!! (BOTH UP AND DOWN)
RE_GENE_NAME = re.compile(r'Name=(?P<gene_name>.+?);') 
#|Below| Searching for data in attributes col and then creating new col for that data
def extract_gene_name(attributes_str): 
    res = RE_GENE_NAME.search(attributes_str)
    return res.group('gene_name')
geneAnnotations['gene_symbol'] = geneAnnotations.attributes.apply(extract_gene_name)

In [26]:
RE_GENE_ID = re.compile(r'gene_id=(?P<gene_id>ENSG.+?);')
def extract_gene_id(attributes_str): # New COL again
    res = RE_GENE_ID.search(attributes_str)
    return res.group('gene_id')
geneAnnotations['gene_ensID'] = geneAnnotations.attributes.apply(extract_gene_id)

In [27]:
geneAnnotations['length'] = geneAnnotations.end - geneAnnotations.start + 1

In [28]:
geneAnnotationsDF = geneAnnotations.drop(['attributes'], axis=1)
chromosomesDF = chromosome.drop(['attributes'], axis=1)

# <font color=#4286f4>Creating DataFrame for n Chromosomes with n sites </font>

In [29]:
#I need random chromosome combined with random position (Somewhere between start and end)
def nBindingsites(n):    
    randChr = chromosomesDF.sample(n, replace=True)
    randChr['BindingSite'] = (np.random.uniform(1, randChr.end).round()).astype(int)
    return randChr
    #Returns a DataFrame with Random sample number with random BindingSite

In [30]:
#MIGHT NOT NEED ANY OF THIS ACTUALLY COULD BE FASTER 

# Distance from Body
def distanceBody(start, end, site):
    return min(abs(start - site), abs(end - site))

# Distance from TSS
def distanceTSS(start, site):
    return abs(start - site)

#Returns True or False to wheather the binding site is within the window size to Body | Start Site below
def isNearBody(start, end, site, window):
    if abs(start - site) <= window:
        return True
    elif abs(end - site) <= window:
        return True
    else:
        return False 
    
def isNearTSS(start, site, window):
    if abs(start - site) <= window:
        return True
    else: 
        return False
#-------------------------------------------- ## Should there be anymore iterations? Checker function 
def inRange(start, site, window):
    if start > (site + window):
        return False 
    else:
        return True

# For: k = Infinity 

In [31]:
#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(bodies) from its site.
def genesInWindowBody(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearBody(row['start'], row['end'], site, window):
            count = count + 1
            geneL.append(row['gene_symbol'])
    return(count, geneL)

#Given specific chromosome, rdm binding site, window: Will calculate number of nearby gene(TSS) from the site.
def genesInWindowTSS(chrNum, site, window):
    count = 0
    geneL = []
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearTSS(row['start'], site, window):
            count = count + 1
            geneL.append(row['gene_symbol'])    
    return(count, geneL)

## <font color=#4286f4>Given a window: Find all the genes inside window for the dataframe of chromosomes</font>

In [32]:
# Takes DF and Window size: Returnes Total Gene count and List of Genes Near Body of GENE (K=INFINITY)
def geneCountNsitesBody(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowBody(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# Takes DF and Window size: Returnes Total Gene count and List of Genes Near TSS of GENE (K=INFINITY)
def geneCountNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        c = genesInWindowTSS(row['chromosome'], row['BindingSite'], window)
        count = count + c[0]
        geneL.append(c[1])
    return(count, geneL)

# For: k = 1

In [33]:
#Given specific chromosome, rdm binding site, window: Will calculate closest gene(body) from its site. and distance
def closestGeneBody(chrNum, site, window):
    geneDF = geneAnnotationsDF.loc[geneAnnotationsDF['chromosome'] == str(chrNum)]
    
    startG = geneDF.iloc[(geneDF['start'] - site).abs().argsort()].head(1)
    endG = geneDF.iloc[(geneDF['end'] - site).abs().argsort()].head(1)
    
    startDist = distanceBody(startG['start'].iloc[0], startG['end'].iloc[0], site)
    endDist = distanceBody(endG['start'].iloc[0], endG['end'].iloc[0], site)
    
    if startDist < endDist:
        if isNearBody(startG['start'].iloc[0], startG['end'].iloc[0], site, window):
            return(startG['gene_symbol'].iloc[0], startDist)
        else:
            return('No Result')
    else:
        if isNearBody(endG['start'].iloc[0], endG['end'].iloc[0], site, window):
            return(endG['gene_symbol'].iloc[0], endDist)
        else:
            return('No Result')

#Given specific chromosome, rdm binding site, window: Will calculate closest gene(TSS) from its site. and Distance
def closestGeneTSS(chrNum, site, window):
    out = ['No Result', 9**90]
    geneDF = geneAnnotations.loc[geneAnnotations['chromosome'] == str(chrNum)]
    
    for index, row in geneDF.iterrows():
        if isNearTSS(row['start'], site, window):
            if distanceTSS(row['start'], site) < out[1]:
                out[0] = row['gene_symbol']
                out[1] = distanceTSS(row['start'], site)
    # Returns closest gene and its distance from TSS. 
    return(out)

In [34]:
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its BODY for every site(K = 1)
def closestGenesNsitesBody(df, window):
    count = 0 
    geneL = []
    
    for row in zip(df['chromosome'].astype(str), df['BindingSite'].values):
        x = closestGeneBody(row[0], row[1], window)
        if x == 'No Result':
            pass
        else:
            geneL.append(x[0])
            count = count + 1
    return(count, geneL)
    
# Takes DF and Window size: Returnes Total Gene count and List of CLOSEST Gene Near its TSS for every site (K = 1)
def closestGenesNsitesTSS(df, window):
    count = 0 
    geneL = []
    
    for index, row in df.iterrows():
        x = closestGeneTSS(row['chromosome'], row['BindingSite'], window)
        geneL.append(x[0])
        count = count + 1
    return(count, geneL)

In [35]:
# For testing purposes
y = nBindingsites(10000)